In [1]:
from fedhex.train._flows import *
from tensorflow_probability import distributions as tfd

Welcome to JupyROOT 6.30/02


2024-07-16 09:31:20.071617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
len_event = 3
len_cond = 2
num_samples = 1000

rng = np.random.default_rng(seed=0x1ace)
data = rng.normal(0, 1, size=(num_samples, len_event))
cond = rng.uniform(0, 10, size=(num_samples, len_cond))

In [3]:
num_flows = 10

made_factory = MADEFactory(params=2,
                           event_shape=len_event,
                           conditional=True,
                           conditional_event_shape=len_cond,
                           hidden_units=[16, 16])

base = tfd.Sample(tfd.Normal(loc=0., scale=1.), sample_shape=[2])

In [4]:
whiten = Whiten(data=data, at_head=True)

maf = MAFComponent(shift_and_log_scale_fn_factory=made_factory,
                   conditional=True)

perm = Permute(permutation=np.arange(0, len_event, -1)[::-1])
bn = BatchNorm()

builder = whiten + num_flows * (perm + bn + maf)

In [5]:
flow = builder.build(base=base)

In [6]:
flow.compile(optimizer="adam",
             loss=NLL)

In [7]:
flow.fit(x=[data, cond],
         y=np.zeros(num_samples),
         initial_epoch=0,
         epochs=1)

UnboundLocalError: in user code:

    File "/home/asatk/miniconda3/envs/rootflows/lib/python3.11/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/asatk/miniconda3/envs/rootflows/lib/python3.11/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/asatk/miniconda3/envs/rootflows/lib/python3.11/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/home/asatk/miniconda3/envs/rootflows/lib/python3.11/site-packages/keras/src/engine/training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "/home/asatk/miniconda3/envs/rootflows/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph